# Prophesee Event Camera 

In this tutorial we demonstrate how to obtain data from a Prophesee camera and stream the events into Lava. We show the basic usage of the `PropheseeCamera` Process and how to apply filters and transformations.

In [1]:
import matplotlib
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

from lava.magma.core.run_configs import Loihi2SimCfg
from lava.magma.core.run_conditions import RunSteps

from lava.lib.peripherals.dvs.prophesee import PropheseeCamera

from utils import EventVisualizer

## Download raw event recording

Open a RAW event data file, if the file doesn't exist, it will be downloaded from Prophesee's public sample server


In [2]:
import metavision_core as mv

EVENT_RECORDING_FILENAME = "80_balls.raw"
mv.utils.get_sample(EVENT_RECORDING_FILENAME)

reader = mv.event_io.RawReader(EVENT_RECORDING_FILENAME)
height, width = reader.get_size()
del reader

## Basic usage

Initialize `PropheseeCamera` using the path to a recording file if RAW format, in order to use a live camera, replace the path with an empty string ("") or leave it out.

The output of the `PropheseeCamera` is a histogram over all pixels. Optionally, the histogram can span multiple output time bins. In that case, set the `num_output_time_bins` parameter to the desired value.


<table>
<tr>
    <td> <img src="gifs/basic.gif" alt="Drawing" style="height: 250px;"/> </td>
</tr>
</table>

In [3]:

# Initialize Processes
camera = PropheseeCamera(filename=EVENT_RECORDING_FILENAME,
                              sensor_shape=(height, width),
                              num_output_time_bins=1)

event_visualizer = EventVisualizer(shape=camera.s_out.shape)

# Connect
camera.s_out.connect(event_visualizer.s_in)

# Run
num_steps = 200
run_cfg = Loihi2SimCfg()
run_cnd = RunSteps(num_steps=num_steps)

camera.run(condition=run_cnd, run_cfg=run_cfg)
camera.stop()

## Apply filters

As you see, the falling balls in the recording cause a trail of events. In many cases these trails are blurring the objects and are undesired. In order to reduce the trails, we can apply the TrailFilterAlgorithm from the metavision_sdk. See the metavision documentation for more filters and their usage.


<table>
<tr>
    <td> <img src="gifs/filters.gif" alt="Drawing" style="height: 250px;"/> </td>
</tr>
</table>

In [4]:
from metavision_sdk_cv import TrailFilterAlgorithm, ActivityNoiseFilterAlgorithm


filters = [TrailFilterAlgorithm(width=width, height=height, threshold=100000),
           ActivityNoiseFilterAlgorithm(width=width, height=height, threshold=1000),]


# Initialize Processes
camera = PropheseeCamera(filename=EVENT_RECORDING_FILENAME,
                              filters=filters,
                              sensor_shape=(height, width),
                              num_output_time_bins=1)

event_visualizer = EventVisualizer(shape=camera.s_out.shape)

# Connect
camera.s_out.connect(event_visualizer.s_in)

# Run
num_steps = 200
run_cfg = Loihi2SimCfg()
run_cnd = RunSteps(num_steps=num_steps)

camera.run(condition=run_cnd, run_cfg=run_cfg)
camera.stop()

## Apply transformations

Lava-peripherals includes transformations which can be applied to the event data. For example Downsampling, MergePolarities and Mirroring.


<table>
<tr>
    <td> <img src="gifs/transform.gif" alt="Drawing" style="height: 125px;"/> </td>
</tr>
</table>

In [5]:
from lava.lib.peripherals.dvs.transform import Compose, Downsample

transformations = Compose(
    [
        Downsample(factor=0.5),
    ]
)

# Initialize Processes
camera = PropheseeCamera(filename=EVENT_RECORDING_FILENAME,
                              transformations=transformations,
                              sensor_shape=(height, width),
                              num_output_time_bins=1)

event_visualizer = EventVisualizer(shape=camera.s_out.shape)

# Connect
camera.s_out.connect(event_visualizer.s_in)

# Run
num_steps = 200
run_cfg = Loihi2SimCfg()
run_cnd = RunSteps(num_steps=num_steps)

camera.run(condition=run_cnd, run_cfg=run_cfg)
camera.stop()

## Custom transformation and manual output shape

The implementation for transformations is compatible with both, tonic and torchvision transformations. Check out the tonic documentation to see a complete list of available transformations and their usage (https://tonic.readthedocs.io/en/latest/auto_examples/index.html). If you need to use custom transformation, the automatic shape determination would not work. In that case, you can specify the output shape manually.


<table>
<tr>
    <td> <img src="gifs/custom_trans.gif" alt="Drawing" style="height: 250px;"/> </td>
</tr>
</table>

In [6]:
def expand_x_dim(events):
    events['x'] += 500
    return events
    

transformations = Compose(
    [
        expand_x_dim,
    ]
)

# Initialize Processes
camera = PropheseeCamera(filename=EVENT_RECORDING_FILENAME,
                              transformations=transformations,
                              sensor_shape=(height, width),
                              num_output_time_bins=1,
                              out_shape=(1, 2, height, width+500))

event_visualizer = EventVisualizer(shape=camera.s_out.shape)

# Connect
camera.s_out.connect(event_visualizer.s_in)

# Run
num_steps = 200
run_cfg = Loihi2SimCfg()
run_cnd = RunSteps(num_steps=num_steps)

camera.run(condition=run_cnd, run_cfg=run_cfg)
camera.stop()